# Cryptography Basics

## Modular Arithmetic

$x$ and $x'$ have the same remainder when divided by $N$:

$$x = x' \bmod N$$

if and only if $N$ devides $x - x'$, i.e. if $\frac{a − b}{n}$ has a remainder of $0$.

The **bracket notation** denotes the **remainder** when $x$ divided by $N$:

$$[x \bmod N]$$

i.e. the unique value $x' \in \{0, \ldots, N-1\}$ such that $x = x' \bmod N$

---

A number $x \bmod N$ is the equivalent of asking for the remainder of $x$ when divided by $N$. 

Two integers $a$ and $b$ are said to be **congruent** (or in the same equivalence class) modulo $N$ if they have the same remainder upon division by $N$. In such a case, we say that 

$$a \equiv b \pmod N$$

<br>

### Addition in Modular Arithmetic

1. If $a + b = c$, then $a \pmod N + b \pmod N \equiv c \pmod N$

2. If $a \equiv b \pmod N$, then $a + k \equiv b + k \pmod N$ for any integer $k$

3. If $a \equiv b \pmod N$ and $c \equiv d \pmod N$, then $a + c \equiv b + d \pmod N$

4. If $a \equiv b \pmod N$, then $-a \equiv -b \pmod N$

<br>

### Multiplication in Modular Arithmetic

1. If $a \cdot b = c$, then $a \pmod N \cdot b \pmod N \equiv c \pmod N$

2. If $a \equiv b \pmod N$, then $k \cdot a \equiv k \cdot b \pmod N$ for any integer $k$

3. If $a \equiv b \pmod N$ and $c \equiv d \pmod N$, then $a \cdot c \equiv b \cdot d \pmod N$

<br>

### Exponentiation in Modular Arithmetic

1. If $a \equiv b \pmod N$, then $a^k \equiv b^k \pmod N$ for any integer $k$

<br>

### Division in Modular Arithmetic

1. If $\gcd(k, N) = 1$ and $k \cdot a \equiv k \cdot b \pmod N$, then $a \equiv b \pmod N$

This property is true, because if $k \cdot (a−b)$ is a multiple of $N$ and $\gcd(k, N) = 1$, then $N$ must divide $a-b$, or equivalently $a \equiv b \pmod N$.

**Example**: Consider $4 \equiv 8 \pmod 4$. Note that we cannot simply divide both sides of the equation by $2$, since $2 \not \equiv 4 \pmod 4$.

<br>

### Multiplicative Inverse in Modular Arithmetic

If $a$ and $N$ are integers such that $\gcd(a, N) = 1$ (coprime or relatively prime), then there exists an integer $b$ such that $a \cdot b \equiv 1 \pmod N$. $b$ is called the multiplicative inverse of $a \bmod N$.

**Examples**:

* $2 \cdot 3 \equiv 1 \pmod{5}$
* $2 \cdot 6 \equiv 1 \pmod{11}$

> TODO: https://en.wikipedia.org/wiki/Modular_arithmetic#Residue_systems

In [1]:
# reference: https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Pseudocode
def mod_inv(x, m):
    r0, r1 = x, m
    s0, s1 = 1, 0
    while r1 != 0:
        quotient = r0 // r1
        r0, r1 = r1, r0 - quotient * r1
        s0, s1 = s1, s0 - quotient * s1
    if r0 != 1:
        return None
    return s0 if s0 >= 0 else s0 + m

In [2]:
mod_inv(2, 11)

6

### Restklassenring

> TODO: https://de.wikipedia.org/wiki/Restklassenring

## Cryptography Principles

### Kerckhoffs's Principle

TODO

### Key Space

The **key space** should be large enough to prevent **brute-force**, exhaustive-search attacks.

## Core Principles of Modern Cryptography

1. **Formal Definitions**: precise mathematical model and definition of what security means

2. **Assumptions**: clearly stated and unambigious

3. **Proofs of Security**: move away from design-break-patch

### Definition of Perfect Secrecy

> "Regardless of any **prior** information the attacker has about the plaintext, the ciphertext should leak no **additional** information about the plaintext"

# Encryption Schemes

A encryption scheme is defined by a **Message Space** $M = M(Gen, Enc_k, Dec_k)$ with the algorithms:

1. **Key Generation** $Gen$ which generates a **key** $k$,

2. **Encryption** $Enc_k$ which takes key $k$ and **message** $m \in M$ as input and outputs **ciphertexst** $c$

$$c \gets \text{Enc}_k (m)$$

3. **Decryption** $Dec_k$ which takes key $k$ and ciphertext $c$ as input and outputs message $m$ or "error"

$$m := \text{Dec}_k (c)$$

and

$$\text{Dec}_k(\text{Enc}_k(m)) = m$$

The **Message Space** $\text{M}$ is the set of all possible messages.

The **Key Space** $\text{K}$ is the set of all possible keys.

The **Ciphertext Space** $\text{C}$ is the set of all possible ciphertexts.

### Notes on the Notation

* The **left arrow** $\gets$ notation denotes assignment to the output of an algorithm that might be **randomized**. Meaning that the output of the algorithm may be different, even when run twice on the same set of inputs.

* The **colon equals** $:=$ denotes an assignment to the output of a **deterministic** algorithm. 

* A single **equal sign** $=$ denotes mathematical equality in contrast to an assignment.

## Probability Distributions

Let $M$ be a **random variable** denoting the value of a message from the **message space**.

This reflects the likelyhood of different messages sent by the parites, given the attackers prior knowledge, e.g.

$$
\begin{align}
\text{P}[M = \text{attack today}] &= 0.7 \\
\text{P}[M = \text{do not attack}] &= 0.3 \\
\end{align}
$$

Let $K$ be a **random variable** denoting the key from the **key space** $\text{M}$.

Given a encryption scheme $M(Gen, Enc_k, Dec_k)$, then $Gen$ defines a **probability distribution** for $K$:

$$
\text{P}[K = k] = \text{P}[\text{Gen outputs key } k]
$$

For example, consider the shift cipher where the key space is $\{0, \ldots, 25\}$, then $\text{P}[K = 15] = 1/26$ 

Here we make the reasonable **assumption**, that $M$ and $K$ are **independent** variables, i.e. the message that a party sends does not depend on the key used to encrypt the message.

Given an encryption scheme $(Gen, Enc_k, Dec_k)$ and a distribution $M$, consider a choosen message $m$ according the given distribution, this then defines a **distribution of the ciphertext**.

Let $C$ be the random variable denoting the value of the ciphertext.

### Example 1

Given a **shift cipher**, then for all $k \in \{0, \ldots, 25\}$ we have $P[K=k] = 1/26$. Say we have $P[M=\text{'}a\text{'}]=0.7$ and $P[M=\text{'}z\text{'}]=0.3$.

What is $P[C=\text{'}b\text{'}]$?

&rarr; The ciphertext $C$ can only be $\text{'}b\text{'}$, if $M=\text{'}a\text{'}$ and $K=1$ or $M=\text{'}z\text{'}$ and $K=2$.

$$
\begin{align}
P[C=\text{'}b\text{'}] &= P[M=\text{'}a\text{'}] \cdot P[K=1] + P[M=\text{'}z\text{'}] \cdot P[K=2] \\[7pt]
&= 0.7 \cdot 1/26 + 0.3 \cdot 1/26 \\[7pt]
&= 1/26 \\
\end{align}
$$

### Example 2

Consider a **Shift cipher** and the distribution $P[M=\text{'}one\text{'}]=0.5$ and $P[M=\text{'}ten\text{'}]=0.5$.

What is $P[C=\text{'}rqh\text{'}]$?

&rarr;

$$
\begin{align}
P[C=\text{'}rqh\text{'}] &= P[C=\text{'}rqh\text{'} \, | \, M=\text{'}one\text{'}] \cdot P[M=\text{'}one\text{'}] + P[C=\text{'}rqh\text{'} \, | \, M=\text{'}ten\text{'}] \cdot P[M=\text{'}ten\text{'}]\\[7pt]
&= 0.5 \cdot 1/26 + 0 \cdot 1/26 \\[7pt]
&= 1/52 \\
\end{align}
$$